In [ ]:
import pandas as pd
import numpy as np
import regex as re

Our team had an issue where people were assigning new user stories to a completely different Jira board via their active Sprint selection, which was causing some issues on the client side. Since I do not have admin privileges with the Jira board, I decided to write a function where I take a csv file and board number as parameters. Once these inputs are provided, the program can return the names of the Reporters for the misaligned user stories. This allows me to quickly track the misaligned stories by using the names returned.

In [ ]:
def board_check(jira_file, board):
    
    # converting csv file to df
    jira_df = pd.read_csv(jira_file)
    
    #drop empty records under Sprint
    jira_df.dropna(axis=0, subset=('Sprint', ), inplace=True)
    
    #change datatype of column to string
    jira_df['Sprint'] = jira_df['Sprint'].astype('string') 
    
    #run check on whether the user story is place in the right board for active sprint
    names = jira_df.loc[jira_df['Sprint'].str.contains(board)==False, 'Reporter']
    return names

In [ ]:
board_check(jira_file, board)

The next function is to find individuals in the team who have created or were assigned user stories. Our project has 90 active members and it has been difficult to keep an account of all users being assigned or creating stories. Sometimes the user is not aware of the different sections that they have to complete when creating a user story. In this function, the parameters include the current Jira csv file and the project roster also in csv format and gives us a list of people from the project who have active user stories and those that do not. Jira currently does not have the capability of filtering for unique users.

In [ ]:
def active_users(jira_file, current_project_mem_file):
        
    # converting csv files to df
    jira_df = pd.read_csv(jira_file)
    current_mem_df = pd.read_csv(current_project_mem_file)
    
    # combining user names under the creator, assignee and reporter column
    # creating set to only list unique names since jira does not have the distinct feature
    creator_list = (jira_df['Creator'].unique()).tolist()
    assignee_list = (jira_df['Assignee'].unique()).tolist()
    reporter_list = (jira_df['Reporter'].unique()).tolist()
    combined_jira_lists = creator_list + assignee_list + reporter_list
    jira_user_set = set(combined_jira_lists)
    
    # setting the names in current_mem_df as set to run intersection 
    current_project_names = set(current_mem_df['Name'].unique())
    
    # set intersection to figure out which current project members also have active jira stories
    active_user = list(current_project_names.intersection(jira_user_set))
    active_user_set = set(active_user)
    mem_not_jira = list(current_project_names.difference(active_user_set))
    jira_not_mem = list(jira_user_set.difference(active_user_set))
    
    print(f"Current project members with Jira stories: {active_user}\n\nCurrent members not using Jira: {mem_not_jira}\n\nJira usernames not in current project: {jira_not_mem}")

In [ ]:
active_users(jira_file, current_project_mem_file)


The above active_members method is not fool proof since some members of the current project may have their names look different on the Jira board, which is why I also print out the leftover names to check with our current list of project members. Mmbers may have the first initial of their 1st name and their last name together. In the next cell I have a regex utilizing code snippet that can be added to the above function to accomodate those individuals, I have commented this out. You can then compare the last names instead if you user list, which in my case has all unique last names. 

In [ ]:
#eg: MSarker would split to 'M Sarker'
#for i in range(len(combined_jira_list)):
    #if ' ' not in creator_list[i]:
        #combined_jira_list[i] = ' '.join(re.findall('[A-Z][^A-Z]*', combined_jira_list[i])) 
#combined_jira_list